In [39]:
#!pip install keras==2.12.0


In [40]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import holidays

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [41]:
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [43]:
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, classification_report, mean_squared_error
from sklearn.calibration import CalibratedClassifierCV, calibration_curve


In [44]:
#Set parameters to see all data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [45]:
# Read dataset

%%time
data_mart_path = 'Enrian_DS_Assignment_Data.csv'

try:
    data_mart_df = pd.read_csv(data_mart_path)
except pd.errors.ParserError as e:
    print(f'Error while parsing CSV file: {e}')



data_mart_df.shape

CPU times: user 539 ms, sys: 43.4 ms, total: 583 ms
Wall time: 1.69 s


(381109, 12)

In [46]:
# Method for reducing the memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [47]:
# Replace uppercase characters of dataframe column names with lowercase
data_mart_df.columns = data_mart_df.columns.str.lower()

In [48]:
# Create a new column 'is_female' with boolean values
data_mart_df['is_female'] = data_mart_df['gender'] == 'Female'
# Convert boolean values to 1 for True and 0 for False
data_mart_df['is_female'] = data_mart_df['is_female'].astype(int)
# Drop the 'gender' column from the DataFrame
data_mart_df.drop(columns=['gender'], inplace=True)

In [49]:
# Create a new column 'is_damage' with boolean values
data_mart_df['is_damage'] = data_mart_df['vehicle_damage'] == 'Yes'
# Convert boolean values to 1 for True and 0 for False
data_mart_df['is_damage'] = data_mart_df['is_damage'].astype(int)
# Drop the 'vehicle_damage' column from the DataFrame
data_mart_df.drop(columns=['vehicle_damage'], inplace=True)

In [50]:
# Define a mapping dictionary
mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}

# Map the values in the 'vehicle_age' column to integers
data_mart_df['vehicle_age'] = data_mart_df['vehicle_age'].map(mapping)

# Display the updated DataFrame
data_mart_df.head()

,id,age,driving_license,region_code,previously_insured,vehicle_age,annual_premium,policy_sales_channel,tenure,response,is_female,is_damage
0,1,44,1,28,0,2,487,26,217,1,0,1
1,2,76,1,3,0,1,404,26,183,0,0,0
2,3,47,1,28,0,2,461,26,27,1,0,1
3,4,21,1,11,1,0,345,152,203,0,0,0
4,5,29,1,41,1,0,331,152,39,0,1,0


In [51]:
data_mart_df.driving_license.value_counts()

driving_license
1    380297
0       812
Name: count, dtype: int64

In [52]:
def replace_cat_info_to_count(df, col):
    # Create a dictionary with the counts of each region code
    count_dict = df[col].value_counts().to_dict()

    # Replace region code with repeating number of that region code
    df[col] = df[col].map(count_dict)

    return df, count_dict

In [53]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

In [54]:
# Method for normalization to numeric columns
def normalize_data(dataframe, columns):

    normalized_dataframe = dataframe.copy()

    # Initialize MinMaxScaler
    #scaler = MinMaxScaler()

    # Check if the specified columns exist in the DataFrame
    existing_columns = [col for col in columns if col in normalized_dataframe.columns]

    if existing_columns:
        # Fit and transform the specified columns using MinMaxScaler
        normalized_dataframe[existing_columns] = scaler.fit_transform(normalized_dataframe[existing_columns])

    return normalized_dataframe

In [55]:
get_scaler_cols = ['vehicle_age', 'annual_premium', 'tenure']

data_mart_df = normalize_data(data_mart_df, get_scaler_cols)
data_mart_df[get_scaler_cols].head()

,vehicle_age,annual_premium,tenure
0,1.0,0.070259,0.716263
1,0.5,0.057443,0.598616
2,1.0,0.066245,0.058824
3,0.0,0.048332,0.667820
4,0.0,0.046170,0.100346


In [56]:
#data_mart_df = reduce_mem_usage(data_mart_df)

In [57]:
X = data_mart_df.drop(['response'], axis=1) #features (independent variables)
y = data_mart_df['response'] #target (dependent variable)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((304887, 11), (76222, 11), (304887,), (76222,))

In [60]:
y_test.values

array([0, 0, 0, ..., 0, 0, 0])

In [61]:
result_df = pd.DataFrame()
result_df['id'] = X_test['id'].copy()
result_df['actual_response'] = y_test.values
result_df.head()

,id,actual_response
200222,200223,0
49766,49767,0
172201,172202,0
160713,160714,0
53272,53273,0


In [62]:
result_df.head()

,id,actual_response
200222,200223,0
49766,49767,0
172201,172202,0
160713,160714,0
53272,53273,0


In [63]:
X_train['age'].min(), X_train['age'].mean()

(20, 38.835581707321076)

In [64]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304887 entries, 332803 to 121958
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    304887 non-null  int64  
 1   age                   304887 non-null  int64  
 2   driving_license       304887 non-null  int64  
 3   region_code           304887 non-null  int64  
 4   previously_insured    304887 non-null  int64  
 5   vehicle_age           304887 non-null  float64
 6   annual_premium        304887 non-null  float64
 7   policy_sales_channel  304887 non-null  int64  
 8   tenure                304887 non-null  float64
 9   is_female             304887 non-null  int64  
 10  is_damage             304887 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 27.9 MB


In [65]:
# Create bins for 'age'
bins = [X_train['age'].min(),
        X_train['age'].mean() - X_train['age'].std(),
        X_train['age'].mean(),
        X_train['age'].mean() + X_train['age'].std(),
        X_train['age'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(X_train['age'], bins=bins)

# Create groups with bins
data_set_size = X_train.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_set_size)

Bins and sizes:
                age   count
0    (20.0, 23.326]   49345
1  (23.326, 38.836]  107937
2  (38.836, 54.345]   89202
3    (54.345, 85.0]   53434


In [66]:
# Create bins for 'age'
age_bin_ranges = [X_train['age'].min(),
                  X_train['age'].mean() - X_train['age'].std(),
                  X_train['age'].mean(),
                  X_train['age'].mean() + X_train['age'].std(),
                  X_train['age'].max()]
X_train['age'] = pd.cut(X_train['age'],
                        bins=age_bin_ranges,
                        include_lowest=True,
                        duplicates='drop').cat.codes
X_train['age'] = scaler.fit_transform(X_train['age'].values.reshape(-1, 1))

X_test['age'] = pd.cut(X_test['age'],
                        bins=age_bin_ranges,
                        include_lowest=True,
                        duplicates='drop').cat.codes
X_test['age'] = scaler.fit_transform(X_test['age'].values.reshape(-1, 1))

X_train['age'].head(10)

332803    0.666667
116248    0.333333
255005    0.000000
317474    0.000000
344212    1.000000
26229     0.333333
109099    0.666667
318077    0.000000
104641    0.000000
107337    1.000000
Name: age, dtype: float64

In [67]:
# Create bins for 'policy_sales_channel'
bins = [X_train['policy_sales_channel'].min(),
        X_train['policy_sales_channel'].median() - X_train['policy_sales_channel'].std(),
        X_train['policy_sales_channel'].median(),
        #X_train['policy_sales_channel'].median() + X_train['policy_sales_channel'].std(),
        X_train['policy_sales_channel'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(X_train['policy_sales_channel'], bins=bins)

# Create groups with bins
data_set_size = X_train.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_set_size)

Bins and sizes:
  policy_sales_channel   count
0         (1.0, 76.77]   81539
1       (76.77, 131.0]   70154
2       (131.0, 163.0]  152352


In [68]:
# Create bins for 'policy_sales_channel'
policy_sales_channel_bin_ranges = [X_train['policy_sales_channel'].min(),
                                   X_train['policy_sales_channel'].median() - X_train['policy_sales_channel'].std(),
                                   X_train['policy_sales_channel'].median(),
                                   X_train['policy_sales_channel'].max()]
X_train['policy_sales_channel'] = pd.cut(X_train['policy_sales_channel'],
                                         bins=policy_sales_channel_bin_ranges,
                                         include_lowest=True,
                                         duplicates='drop').cat.codes
X_train['policy_sales_channel'] = scaler.fit_transform(X_train['policy_sales_channel'].values.reshape(-1, 1))

X_test['policy_sales_channel'] = pd.cut(X_test['policy_sales_channel'],
                                        bins=policy_sales_channel_bin_ranges,
                                        include_lowest=True,
                                        duplicates='drop').cat.codes
X_test['policy_sales_channel'] = scaler.fit_transform(X_test['policy_sales_channel'].values.reshape(-1, 1))

X_train['policy_sales_channel'].head(10)

332803    0.0
116248    0.0
255005    1.0
317474    1.0
344212    1.0
26229     1.0
109099    0.5
318077    1.0
104641    1.0
107337    0.0
Name: policy_sales_channel, dtype: float64

In [69]:
X_train.head()

,id,age,driving_license,region_code,previously_insured,vehicle_age,annual_premium,policy_sales_channel,tenure,is_female,is_damage
332803,332804,0.666667,1,15,0,0.5,0.093422,0.0,0.750865,1,1
116248,116249,0.333333,1,11,0,0.5,0.037986,0.0,0.065744,0,1
255005,255006,0.000000,1,30,1,0.0,0.079370,1.0,0.539792,0,0
317474,317475,0.000000,1,41,1,0.0,0.049259,1.0,0.923875,1,0
344212,344213,1.000000,1,48,0,1.0,0.000000,1.0,0.501730,0,1


In [70]:
#region_code
X_train, region_count_dict = replace_cat_info_to_count(X_train, 'region_code')
X_train['region_code'] = scaler.fit_transform(X_train['region_code'].values.reshape(-1, 1))

# Replace region code with repeating number of that region code
X_test['region_code'] = X_test['region_code'].map(region_count_dict)
X_test['region_code'] = scaler.fit_transform(X_test['region_code'].values.reshape(-1, 1))

X_train.head()

,id,age,driving_license,region_code,previously_insured,vehicle_age,annual_premium,policy_sales_channel,tenure,is_female,is_damage
332803,332804,0.666667,1,0.123075,0,0.5,0.093422,0.0,0.750865,1,1
116248,116249,0.333333,1,0.084654,0,0.5,0.037986,0.0,0.065744,0,1
255005,255006,0.000000,1,0.113252,1,0.0,0.079370,1.0,0.539792,0,0
317474,317475,0.000000,1,0.169931,1,0.0,0.049259,1.0,0.923875,1,0
344212,344213,1.000000,1,0.042744,0,1.0,0.000000,1.0,0.501730,0,1


In [71]:
#policy_sales_channel
X_train, channel_count_dict = replace_cat_info_to_count(X_train, 'policy_sales_channel')
X_train['policy_sales_channel'] = scaler.fit_transform(X_train['policy_sales_channel'].values.reshape(-1, 1))

# Replace region code with repeating number of that region code
X_test['policy_sales_channel'] = X_test['policy_sales_channel'].map(channel_count_dict)
X_test['policy_sales_channel'] = scaler.fit_transform(X_test['policy_sales_channel'].values.reshape(-1, 1))

X_train.head()

,id,age,driving_license,region_code,previously_insured,vehicle_age,annual_premium,policy_sales_channel,tenure,is_female,is_damage
332803,332804,0.666667,1,0.123075,0,0.5,0.093422,0.148751,0.750865,1,1
116248,116249,0.333333,1,0.084654,0,0.5,0.037986,0.148751,0.065744,0,1
255005,255006,0.000000,1,0.113252,1,0.0,0.079370,1.000000,0.539792,0,0
317474,317475,0.000000,1,0.169931,1,0.0,0.049259,1.000000,0.923875,1,0
344212,344213,1.000000,1,0.042744,0,1.0,0.000000,1.000000,0.501730,0,1


In [72]:
X_train.drop(columns=['id'], inplace=True)
X_test.drop(columns=['id'], inplace=True)

In [73]:
X_test.head()

,age,driving_license,region_code,previously_insured,vehicle_age,annual_premium,policy_sales_channel,tenure,is_female,is_damage
200222,0.000000,1,0.085030,1,0.0,0.033045,1.000000,0.214533,1,0
49766,1.000000,1,0.123075,0,0.5,0.064855,0.148751,0.318339,0,1
172201,0.666667,1,0.085030,0,0.5,0.000000,0.148751,0.276817,1,1
160713,0.333333,1,0.084654,0,0.0,0.000000,1.000000,0.892734,1,0
53272,0.666667,1,0.010504,0,0.5,0.058369,0.000000,0.882353,0,1


In [74]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((304887, 10), (76222, 10), (304887,), (76222,))

##XGB Regressor

In [75]:
# Define XGBRegressor

%%time
xgb = XGBRegressor(enable_categorical=True)

# Define parameter grid for grid search
param_grid = { 'gamma' : [0.3, 0.1], 'learning_rate' : [0.01, 0.015],
              'max_depth' : [2, 5],
              'n_estimators' : [150, 250],
              'nthread' : [-1], 'reg_alpha' : [1], 'reg_lambda' : [1], 'seed' : [10]
              }

# Perform grid search with time series cross-validation
#tscv = TimeSeriesSplit(n_splits=-1)
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, verbose=True)
grid_search.fit(X_train, y_train, eval_set=[(X_train, y_train)], early_stopping_rounds=50)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:0.32679
[1]	validation_0-rmse:0.32635
[2]	validation_0-rmse:0.32591
[3]	validation_0-rmse:0.32547
[4]	validation_0-rmse:0.32505
[5]	validation_0-rmse:0.32463
[6]	validation_0-rmse:0.32423
[7]	validation_0-rmse:0.32383
[8]	validation_0-rmse:0.32343
[9]	validation_0-rmse:0.32304
[10]	validation_0-rmse:0.32266
[11]	validation_0-rmse:0.32229
[12]	validation_0-rmse:0.32192
[13]	validation_0-rmse:0.32156
[14]	validation_0-rmse:0.32121
[15]	validation_0-rmse:0.32086
[16]	validation_0-rmse:0.32052
[17]	validation_0-rmse:0.32018
[18]	validation_0-rmse:0.31985
[19]	validation_0-rmse:0.31953
[20]	validation_0-rmse:0.31921
[21]	validation_0-rmse:0.31890
[22]	validation_0-rmse:0.31859
[23]	validation_0-rmse:0.31829
[24]	validation_0-rmse:0.31799
[25]	validation_0-rmse:0.31770
[26]	validation_0-rmse:0.31742
[27]	validation_0-rmse:0.31714
[28]	validation_0-rmse:0.31686
[29]	validation_0-rmse:0.31659
[30]	validation_0-rmse:0.31632
[31]	validation_0-rmse:0.31606
[32]	validation_0-

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[2]	validation_0-rmse:0.32590
[3]	validation_0-rmse:0.32546
[4]	validation_0-rmse:0.32504
[5]	validation_0-rmse:0.32462
[6]	validation_0-rmse:0.32421
[7]	validation_0-rmse:0.32381
[8]	validation_0-rmse:0.32341
[9]	validation_0-rmse:0.32302
[10]	validation_0-rmse:0.32264
[11]	validation_0-rmse:0.32226
[12]	validation_0-rmse:0.32189
[13]	validation_0-rmse:0.32153
[14]	validation_0-rmse:0.32117
[15]	validation_0-rmse:0.32082
[16]	validation_0-rmse:0.32048
[17]	validation_0-rmse:0.32014
[18]	validation_0-rmse:0.31981
[19]	validation_0-rmse:0.31948
[20]	validation_0-rmse:0.31916
[21]	validation_0-rmse:0.31885
[22]	validation_0-rmse:0.31854
[23]	validation_0-rmse:0.31824
[24]	validation_0-rmse:0.31794
[25]	validation_0-rmse:0.31765
[26]	validation_0-rmse:0.31736
[27]	validation_0-rmse:0.31708
[28]	validation_0-rmse:0.31680
[29]	validation_0-rmse:0.31653
[30]	validation_0-rmse:0.31626
[31]	validation_0-rmse:0.31600
[32]	validation_0-rmse:0.31574
[33]	validation_0-rmse:0.31549
[34]	validation_

...

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[2]	validation_0-rmse:0.32488
[3]	validation_0-rmse:0.32411
[4]	validation_0-rmse:0.32335
[5]	validation_0-rmse:0.32262
[6]	validation_0-rmse:0.32191
[7]	validation_0-rmse:0.32124
[8]	validation_0-rmse:0.32059
[9]	validation_0-rmse:0.31993
[10]	validation_0-rmse:0.31930
[11]	validation_0-rmse:0.31868
[12]	validation_0-rmse:0.31810
[13]	validation_0-rmse:0.31753
[14]	validation_0-rmse:0.31696
[15]	validation_0-rmse:0.31640
[16]	validation_0-rmse:0.31586
[17]	validation_0-rmse:0.31536
[18]	validation_0-rmse:0.31485
[19]	validation_0-rmse:0.31437
[20]	validation_0-rmse:0.31389
[21]	validation_0-rmse:0.31343
[22]	validation_0-rmse:0.31300
[23]	validation_0-rmse:0.31255
[24]	validation_0-rmse:0.31214
[25]	validation_0-rmse:0.31172
[26]	validation_0-rmse:0.31132
[27]	validation_0-rmse:0.31093
[28]	validation_0-rmse:0.31055
[29]	validation_0-rmse:0.31018
[30]	validation_0-rmse:0.30983
[31]	validation_0-rmse:0.30947
[32]	validation_0-rmse:0.30914
[33]	validation_0-rmse:0.30882
[34]	validation_

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-rmse:0.32200
[7]	validation_0-rmse:0.32133
[8]	validation_0-rmse:0.32068
[9]	validation_0-rmse:0.32004
[10]	validation_0-rmse:0.31943
[11]	validation_0-rmse:0.31880
[12]	validation_0-rmse:0.31820
[13]	validation_0-rmse:0.31760
[14]	validation_0-rmse:0.31705
[15]	validation_0-rmse:0.31652
[16]	validation_0-rmse:0.31597
[17]	validation_0-rmse:0.31544
[18]	validation_0-rmse:0.31495
[19]	validation_0-rmse:0.31447
[20]	validation_0-rmse:0.31399
[21]	validation_0-rmse:0.31353
[22]	validation_0-rmse:0.31307
[23]	validation_0-rmse:0.31265
[24]	validation_0-rmse:0.31223
[25]	validation_0-rmse:0.31181
[26]	validation_0-rmse:0.31141
[27]	validation_0-rmse:0.31103
[28]	validation_0-rmse:0.31064
[29]	validation_0-rmse:0.31028
[30]	validation_0-rmse:0.30991
[31]	validation_0-rmse:0.30957
[32]	validation_0-rmse:0.30924
[33]	validation_0-rmse:0.30890
[34]	validation_0-rmse:0.30859
[35]	validation_0-rmse:0.30828
[36]	validation_0-rmse:0.30797
[37]	validation_0-rmse:0.30767
[38]	validat

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=True, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, ma...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'gamma': [0.3, 0.1], 'learning_rate': [0.01, 0.015],
                         'max_depth': [2, 5], 'n_estimators': [150, 250],
                         'nthread': [-1], 'reg_alpha': [1], 'reg_lambda': [1],
                         'seed': [10]},
             verbose=True)

In [76]:
grid_search.best_params_

{'gamma': 0.1,
 'learning_rate': 0.015,
 'max_depth': 5,
 'n_estimators': 250,
 'nthread': -1,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'seed': 10}

In [77]:
grid_search.best_score_

0.17888329915378565

In [78]:
# Define custom MAPE scorer
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = y_true.astype(np.int32)
    y_pred = y_pred.astype(np.int32)

    # Calculate absolute percentage error
    absolute_percentage_error = np.abs((y_true - y_pred) / y_true)

    # Handle division by zero (replace with zero)
    absolute_percentage_error = absolute_percentage_error[~np.isnan(absolute_percentage_error)]  # Remove NaNs

    # Calculate mean absolute percentage error
    mape = np.mean(absolute_percentage_error) * 100
    return mape

In [79]:
%%time
# Predict with X_test data
y_xgb_prob = grid_search.predict(X_test)

# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(y_test.astype(np.int16), y_xgb_prob.astype(np.int16)))
mape = mean_absolute_percentage_error(y_test, y_xgb_prob)

print("RMSE:", rmse)
print("MAPE:", mape)


RMSE: 0.3534652486936423
MAPE: 100.0
CPU times: user 279 ms, sys: 1.99 ms, total: 281 ms
Wall time: 278 ms


In [80]:
y_xgb_prob

array([0.003182  , 0.18981202, 0.2967678 , ..., 0.003182  , 0.003182  ,
       0.003182  ], dtype=float32)

In [107]:
rounded_arr = np.where(y_xgb_prob > 0.5, np.ceil(y_xgb_prob), np.floor(y_xgb_prob))

print(classification_report(y_test, rounded_arr))


              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       0.88      1.00      0.93     66699
         1.0       0.71      0.00      0.00      9523

    accuracy                           0.87     76222
   macro avg       0.53      0.33      0.31     76222
weighted avg       0.85      0.87      0.82     76222



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [81]:
result_df['xgb_prediction'] = y_xgb_prob
result_df.head(10)

,id,actual_response,xgb_prediction
200222,200223,0,0.003182
49766,49767,0,0.189812
172201,172202,0,0.296768
160713,160714,0,0.066643
53272,53273,0,0.270498
372603,372604,0,0.003182
216160,216161,0,0.297581
59206,59207,0,0.003182
26462,26463,0,0.181835
95043,95044,1,0.290471


# RandomForestClassifier



In [82]:
n_estimators = [700]
max_depth = [10, 8]
min_samples_split = [10, 5]
min_samples_leaf = [5, 3]

In [83]:
hyper_random = {"n_estimators":n_estimators,
                "max_depth":max_depth,
                "min_samples_split":min_samples_split,
                "min_samples_leaf":min_samples_leaf}

In [84]:
%%time
clf_rf_tuned = GridSearchCV(RandomForestClassifier(), hyper_random,
                            cv = 5, verbose = 1,
                            n_jobs = -1)
clf_rf_tuned.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


CPU times: user 3min 39s, sys: 8.58 s, total: 3min 48s
Wall time: 1h 18min 27s


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 8], 'min_samples_leaf': [5, 3],
                         'min_samples_split': [10, 5], 'n_estimators': [700]},
             verbose=1)

In [85]:
best_params_random = clf_rf_tuned.best_params_
print(best_params_random)

{'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 700}


In [86]:
CV_clf_rf = RandomForestClassifier(max_depth=best_params_random["max_depth"],
                                   min_samples_leaf=best_params_random["min_samples_leaf"],
                                   min_samples_split=best_params_random["min_samples_split"],
                                   n_estimators= best_params_random["n_estimators"])

In [87]:
%%time
# Train model
CV_clf_rf.fit(X_train, y_train)

# Predict with X_test data
y_test_predict_random = CV_clf_rf.predict_proba(X_test)[:, 1]
yhat_random = CV_clf_rf.predict(X_test)
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_test_predict_random, n_bins=10)

CPU times: user 3min 9s, sys: 191 ms, total: 3min 9s
Wall time: 3min 11s


In [88]:
print(classification_report(y_test, yhat_random))


              precision    recall  f1-score   support

           0       0.88      1.00      0.93     66699
           1       0.75      0.00      0.00      9523

    accuracy                           0.88     76222
   macro avg       0.81      0.50      0.47     76222
weighted avg       0.86      0.88      0.82     76222



In [89]:
# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(y_test.astype(np.int16), y_xgb_prob.astype(np.int16)))
mape = mean_absolute_percentage_error(y_test, y_xgb_prob)

print("RMSE:", rmse)
print("MAPE:", mape)

RMSE: 0.3534652486936423
MAPE: 100.0


In [90]:
result_df['rf_prediction'] = y_test_predict_random


In [91]:
result_df.to_csv("results.csv", index=False)


In [92]:
result_df[result_df.actual_response == 1].head(10)

,id,actual_response,xgb_prediction,rf_prediction
95043,95044,1,0.290471,0.297967
55640,55641,1,0.036425,0.031971
196603,196604,1,0.304423,0.309293
292232,292233,1,0.325832,0.329878
198094,198095,1,0.324188,0.322976
107941,107942,1,0.329538,0.335496
269651,269652,1,0.164270,0.199483
25377,25378,1,0.216502,0.137003
64439,64440,1,0.178245,0.170944
227812,227813,1,0.287089,0.296045


# Naive Bias

In [93]:
%%time
# Uncalibrated
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)
y_test_predict_nb = clf_nb.predict_proba(X_test)[:, 1]
yhat_nb = clf_nb.predict(X_test)
fraction_of_positives_nb, mean_predicted_value_nb = calibration_curve(y_test, y_test_predict_nb, n_bins=10)


CPU times: user 135 ms, sys: 4.02 ms, total: 139 ms
Wall time: 141 ms


In [94]:
print(classification_report(y_test, yhat_nb))


              precision    recall  f1-score   support

           0       0.99      0.64      0.78     66699
           1       0.28      0.95      0.43      9523

    accuracy                           0.68     76222
   macro avg       0.63      0.80      0.60     76222
weighted avg       0.90      0.68      0.74     76222



In [95]:
result_df['nb_prediction'] = y_test_predict_nb
result_df.to_csv("results.csv", index=False)


In [96]:
X_train.head()

,age,driving_license,region_code,previously_insured,vehicle_age,annual_premium,policy_sales_channel,tenure,is_female,is_damage
332803,0.666667,1,0.123075,0,0.5,0.093422,0.148751,0.750865,1,1
116248,0.333333,1,0.084654,0,0.5,0.037986,0.148751,0.065744,0,1
255005,0.000000,1,0.113252,1,0.0,0.079370,1.000000,0.539792,0,0
317474,0.000000,1,0.169931,1,0.0,0.049259,1.000000,0.923875,1,0
344212,1.000000,1,0.042744,0,1.0,0.000000,1.000000,0.501730,0,1


In [97]:
%%time
# Calibrated
clf_sigmoid_nb = CalibratedClassifierCV(clf_nb, cv=10, method='isotonic')
clf_sigmoid_nb.fit(X_train, y_train)

y_test_predict_nb_calib = clf_sigmoid_nb.predict_proba(X_test)[:, 1]

yhat_calibrated_nb = clf_sigmoid_nb.predict(X_test)

fraction_of_positives_nb_calib, mean_predicted_value_nb_calib = calibration_curve(y_test, y_test_predict_nb_calib, n_bins=10)


CPU times: user 1.68 s, sys: 8.96 ms, total: 1.69 s
Wall time: 1.71 s


In [98]:
print(classification_report(y_test, yhat_calibrated_nb))


              precision    recall  f1-score   support

           0       0.88      1.00      0.93     66699
           1       0.27      0.00      0.00      9523

    accuracy                           0.87     76222
   macro avg       0.57      0.50      0.47     76222
weighted avg       0.80      0.87      0.82     76222



In [99]:
result_df['nb_isotonic_prediction'] = y_test_predict_nb_calib
result_df.to_csv("results.csv", index=False)

In [100]:
%%time
# Calibrated, Platt
clf_sigmoid_nb_calib_sig = CalibratedClassifierCV(clf_nb, cv=10, method='sigmoid')
clf_sigmoid_nb_calib_sig.fit(X_train, y_train)

y_test_predict_nb_calib_platt = clf_sigmoid_nb_calib_sig.predict_proba(X_test)[:, 1]
yhat_calibrated_platt = clf_sigmoid_nb_calib_sig.predict(X_test)

fraction_of_positives_nb_calib_platt, mean_predicted_value_nb_calib_platt = calibration_curve(y_test, y_test_predict_nb_calib_platt, n_bins=10)


CPU times: user 2.5 s, sys: 1.31 s, total: 3.8 s
Wall time: 2.35 s


In [101]:
print(classification_report(y_test, yhat_calibrated_platt))


              precision    recall  f1-score   support

           0       0.88      1.00      0.93     66699
           1       0.00      0.00      0.00      9523

    accuracy                           0.88     76222
   macro avg       0.44      0.50      0.47     76222
weighted avg       0.77      0.88      0.82     76222



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [102]:
result_df['nb_sigmoid_prediction'] = y_test_predict_nb_calib_platt
result_df.to_csv("results.csv", index=False)

In [120]:
result_df.head()

,id,actual_response,xgb_prediction,rf_prediction,nb_prediction,nb_isotonic_prediction,nb_sigmoid_prediction
0,200223,0,0.003182,0.000074,2.719684e-77,0.001104,0.010757
1,49767,0,0.189812,0.194077,9.827562e-01,0.277782,0.294929
2,172202,0,0.296768,0.296758,9.793360e-01,0.277782,0.292288
3,160714,0,0.066643,0.045842,9.965050e-11,0.028944,0.010757
4,53273,0,0.270498,0.284367,9.849260e-01,0.277782,0.296607


In [121]:
result_df['nb_prediction'] = result_df['nb_prediction'].apply(lambda x: '{:.5f}'.format(x))
result_df['nb_isotonic_prediction'] = result_df['nb_isotonic_prediction'].apply(lambda x: '{:.5f}'.format(x))
result_df['nb_sigmoid_prediction'] = result_df['nb_sigmoid_prediction'].apply(lambda x: '{:.5f}'.format(x))
result_df['rf_prediction'] = result_df['rf_prediction'].apply(lambda x: '{:.5f}'.format(x))
result_df['xgb_prediction'] = result_df['xgb_prediction'].apply(lambda x: '{:.5f}'.format(x))


result_df.head(10)

,id,actual_response,xgb_prediction,rf_prediction,nb_prediction,nb_isotonic_prediction,nb_sigmoid_prediction
0,200223,0,0.00318,0.00007,0.00000,0.00110,0.01076
1,49767,0,0.18981,0.19408,0.98276,0.27778,0.29493
2,172202,0,0.29677,0.29676,0.97934,0.27778,0.29229
3,160714,0,0.06664,0.04584,0.00000,0.02894,0.01076
4,53273,0,0.27050,0.28437,0.98493,0.27778,0.29661
5,372604,0,0.00318,0.00016,0.00000,0.00110,0.01076
6,216161,0,0.29758,0.34073,0.99307,0.28226,0.30296
7,59207,0,0.00318,0.00012,0.00000,0.00110,0.01076
8,26463,0,0.18183,0.17746,0.97952,0.27778,0.29244
9,95044,1,0.29047,0.29797,0.99407,0.28372,0.30375


In [122]:
result_df.columns

Index(['id', 'actual_response', 'xgb_prediction', 'rf_prediction', 'nb_prediction', 'nb_isotonic_prediction', 'nb_sigmoid_prediction'], dtype='object')

In [124]:
result_df = reduce_mem_usage(result_df)
result_df.info()

Mem. usage decreased to  3.27 Mb (19.6% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76222 entries, 0 to 76221
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      76222 non-null  int32 
 1   actual_response         76222 non-null  int8  
 2   xgb_prediction          76222 non-null  object
 3   rf_prediction           76222 non-null  object
 4   nb_prediction           76222 non-null  object
 5   nb_isotonic_prediction  76222 non-null  object
 6   nb_sigmoid_prediction   76222 non-null  object
dtypes: int32(1), int8(1), object(5)
memory usage: 3.3+ MB


In [127]:
# Convert object-type columns in a DataFrame to float.

def convert_object_to_float(dataframe):

    for column in dataframe.columns:
        # Check if the column dtype is 'object'
        if dataframe[column].dtype == 'object':
            try:
                # Attempt to convert the column to float
                dataframe[column] = dataframe[column].astype(float)
            except ValueError:
                # Print a message if conversion fails
                print(f"Unable to convert column '{column}' to float.")
    return dataframe


In [128]:
result_df = convert_object_to_float(result_df)
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76222 entries, 0 to 76221
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      76222 non-null  int32  
 1   actual_response         76222 non-null  int8   
 2   xgb_prediction          76222 non-null  float64
 3   rf_prediction           76222 non-null  float64
 4   nb_prediction           76222 non-null  float64
 5   nb_isotonic_prediction  76222 non-null  float64
 6   nb_sigmoid_prediction   76222 non-null  float64
dtypes: float64(5), int32(1), int8(1)
memory usage: 3.3 MB


In [129]:
# Method for scoring the predictions
def calc_cancellation_score(final_df, target_col):

    # Get accuracy values
    xgb_accuracy = 0.87
    random_forest_accuracy = 0.88
    naive_bias_accuracy = 0.68
    isotonic_calibrated_naive_bias_accuracy = 0.87
    sigmoid_calibrated_naive_bias_accuracy = 0.88

    final_df[target_col] = (final_df['rf_prediction'] * float(random_forest_accuracy) +
                            final_df['xgb_prediction'] * float(xgb_accuracy) +
                            final_df['nb_prediction'] * float(naive_bias_accuracy) +
                            final_df['nb_isotonic_prediction'] * float(isotonic_calibrated_naive_bias_accuracy) +
                            final_df['nb_sigmoid_prediction'] * float(sigmoid_calibrated_naive_bias_accuracy))

    def NormalizeData(data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))

    final_df[target_col] = NormalizeData(final_df[target_col])

    return final_df

In [130]:
result_df = calc_cancellation_score(result_df, 'response_score')
result_df.head()

,id,actual_response,xgb_prediction,rf_prediction,nb_prediction,nb_isotonic_prediction,nb_sigmoid_prediction,response_score
0,200223,0,0.00318,0.00007,0.00000,0.00110,0.01076,0.004368
1,49767,0,0.18981,0.19408,0.98276,0.27778,0.29493,0.727358
2,172202,0,0.29677,0.29676,0.97934,0.27778,0.29229,0.813974
3,160714,0,0.06664,0.04584,0.00000,0.02894,0.01076,0.062370
4,53273,0,0.27050,0.28437,0.98493,0.27778,0.29661,0.801301


In [137]:
# Binarize the 'response_score' column in the DataFrame based on the given threshold.

def binarize_response_score(df, col, threshold):

    # Apply binarization to 'response_score' column
    df[col] = df['response_score'].apply(lambda x: 1 if x > threshold else 0)

    return df


In [150]:
result_df = binarize_response_score(result_df, 'response_v05', 0.5)
result_df = binarize_response_score(result_df, 'response_v07', 0.7)
result_df = binarize_response_score(result_df, 'response_v09', 0.9)

In [151]:
print('Report for 0.5')

print(classification_report(result_df.actual_response, result_df.response_v05))

print('Report for 0.7')

print(classification_report(result_df.actual_response, result_df.response_v07))

print('Report for 0.9')

print(classification_report(result_df.actual_response, result_df.response_v09))


Report for 0.5
              precision    recall  f1-score   support

           0       0.98      0.69      0.81     66699
           1       0.29      0.89      0.44      9523

    accuracy                           0.71     76222
   macro avg       0.63      0.79      0.62     76222
weighted avg       0.89      0.71      0.76     76222

Report for 0.7
              precision    recall  f1-score   support

           0       0.97      0.70      0.82     66699
           1       0.29      0.86      0.44      9523

    accuracy                           0.72     76222
   macro avg       0.63      0.78      0.63     76222
weighted avg       0.89      0.72      0.77     76222

Report for 0.9
              precision    recall  f1-score   support

           0       0.88      1.00      0.93     66699
           1       0.45      0.03      0.05      9523

    accuracy                           0.87     76222
   macro avg       0.66      0.51      0.49     76222
weighted avg       0.82      

In [152]:
print('Actual 0 case = ', result_df[result_df.actual_response == 0].shape[0])
print('True 0 case = ', result_df[(result_df.actual_response == 0 ) & (result_df.response_v05 == 0)].shape[0])
print('False 0 case = ', result_df[(result_df.actual_response == 0 ) & (result_df.response_v05 == 1)].shape[0])
print('Actual 1 case = ', result_df[result_df.actual_response == 1].shape[0])
print('True 1 case = ', result_df[(result_df.actual_response == 1) & (result_df.response_v05 == 1)].shape[0])
print('False 1 case = ', result_df[(result_df.actual_response == 1) & (result_df.response_v05 == 0)].shape[0])

Actual 0 case =  66699
True 0 case =  45922
False 0 case =  20777
Actual 1 case =  9523
True 1 case =  8502
False 1 case =  1021


In [153]:
print('Actual 0 case = ', result_df[result_df.actual_response == 0].shape[0])
print('True 0 case = ', result_df[(result_df.actual_response == 0 ) & (result_df.response_v07 == 0)].shape[0])
print('False 0 case = ', result_df[(result_df.actual_response == 0 ) & (result_df.response_v07 == 1)].shape[0])
print('Actual 1 case = ', result_df[result_df.actual_response == 1].shape[0])
print('True 1 case = ', result_df[(result_df.actual_response == 1) & (result_df.response_v07 == 1)].shape[0])
print('False 1 case = ', result_df[(result_df.actual_response == 1) & (result_df.response_v07 == 0)].shape[0])

Actual 0 case =  66699
True 0 case =  47020
False 0 case =  19679
Actual 1 case =  9523
True 1 case =  8234
False 1 case =  1289


In [154]:
print('Actual 0 case = ', result_df[result_df.actual_response == 0].shape[0])
print('True 0 case = ', result_df[(result_df.actual_response == 0 ) & (result_df.response_v09 == 0)].shape[0])
print('False 0 case = ', result_df[(result_df.actual_response == 0 ) & (result_df.response_v09 == 1)].shape[0])
print('Actual 1 case = ', result_df[result_df.actual_response == 1].shape[0])
print('True 1 case = ', result_df[(result_df.actual_response == 1) & (result_df.response_v09 == 1)].shape[0])
print('False 1 case = ', result_df[(result_df.actual_response == 1) & (result_df.response_v09 == 0)].shape[0])

Actual 0 case =  66699
True 0 case =  66375
False 0 case =  324
Actual 1 case =  9523
True 1 case =  263
False 1 case =  9260


In [155]:
result_df.to_csv("results.csv", index=False)